In [1]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon_db",
    "user":"root",
    "password":"Youseethis123"
}

try: 
    pool = MySQLConnectionPool(pool_name = "pool_a", pool_size = 2, **dbconfig)
    print("The connection to pool is created with a name: ", pool.pool_name)
    print("The pool size is: ", pool.pool_size)

except Error as er:
    print("Error code: ", er.errno)
    print("Error message: ", er.msg)

The connection to pool is created with a name:  pool_a
The pool size is:  2


In [2]:
connectionP = pool.get_connection()
print("Connection established, now creating a cursor.")
cursor = connectionP.cursor()

Connection established, now creating a cursor.


In [4]:
#Task 2, creating stored procedure
sql_procedure_peak_hours = """
CREATE PROCEDURE PeakHours()
BEGIN
SELECT HOUR(BookingSlot) AS BookingHour, COUNT(HOUR(BookingSlot)) AS TotalperHour
FROM Bookings
GROUP BY BookingHour
ORDER BY TotalperHour DESC;
END
"""

In [5]:
cursor.execute(sql_procedure_peak_hours)

In [6]:
cursor.callproc("PeakHours")

()

In [7]:
results = next(cursor.stored_results())
dataset = results.fetchall()

In [9]:
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]
print(columns)

for data in dataset:
    print(data)

['BookingHour', 'TotalperHour']
(19, 2)
(15, 1)
(17, 1)
(18, 1)
(20, 1)


In [18]:
sql_procedure_guest_status = """
CREATE PROCEDURE GuestStatus()
BEGIN

SELECT CONCAT(GuestFirstName,' ', GuestLastName) AS GuestName,
CASE
WHEN Role IN ('Manager', 'Assistant Manager') THEN "Ready to Pay"
WHEN Role IN ('Head Chef') THEN "Ready to serve"
WHEN Role IN ('Assistant Chef') THEN "Preparing order"
WHEN Role IN ('Head Waiter') THEN "Order served"
ELSE "Please wait..."
END AS Status
FROM Bookings LEFT JOIN Employees ON Bookings.EmployeeID = Employees.EmployeeID;

END
"""

In [19]:
cursor.execute("""DROP PROCEDURE IF EXISTS GuestStatus;""")
cursor.execute(sql_procedure_guest_status)

In [20]:
cursor.callproc("GuestStatus")

()

In [21]:
results = next(cursor.stored_results())
dataset = results.fetchall()
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]
print(columns)

for data in dataset:
    print(data)

['GuestName', 'Status']
('Anna Iversen', 'Ready to Pay')
('Joakim Iversen', 'Ready to Pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing order')
('Hiroki Yamane', 'Ready to Pay')
('Diana Pinto', 'Order served')


In [23]:
#Make sure to close connection
if connectionP.is_connected():
    cursor.close()
    print("The cursor is closed.")
    connectionP.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")

The cursor is closed.
MySQL connection is closed.
